In [56]:
import pandas as pd
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math
import sqlalchemy
import mysql.connector
import meteostat
import pprint
import pickle

In [2]:
Teams_and_IDs = pd.read_csv("Teams_and_IDs.csv")
Player_Positions = pd.read_csv("MLB_Player_Positions.csv")
Park_Data = pd.read_csv("mlb_parks.csv")

In [3]:
All_Teams_Data = []

for team_id in Teams_and_IDs['Team_ID']:
    roster_url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/roster?rosterType=allTime"
    response = requests.get(roster_url)
    roster_data = response.json()['roster']
    for player in roster_data:
        Player_Name = player['person']['fullName']
        Player_ID = player['person']['id']
        Player_Dictionary = {"Team_ID":team_id,"Player Name":Player_Name, "Player ID":Player_ID}
        All_Teams_Data.append(Player_Dictionary)

All_Teams_DataFrame = pd.DataFrame(All_Teams_Data)

In [4]:
# almost the same could be achieve this way.
All_Teams_DataFrame_ = Player_Positions.drop_duplicates(['player_name', 'player_id', 'player_team_id'])[['player_team_id', 'player_name', 'player_id']]

In [5]:
def Team_To_ID(team_name):
    team_id = Teams_and_IDs["Team_ID"][Teams_and_IDs['Team_Name'] == team_name].iloc[0]
    return team_id

def ID_To_Team(team_id):
    team_name = Teams_and_IDs["Team_Name"][Teams_and_IDs['Team_ID'] == team_id].iloc[0]
    return team_name

def Player_to_ID(player_name):
    player_id = Player_Positions["player_id"][Player_Positions["player_name"] == player_name]
    if len(player_id) >= 1:
        return player_id.iloc[0]
    else:
        return ""

def celsius_to_fahrenheit(celsius):
    return ( celsius * (9/5) ) + 32    

## start

In [186]:
tomorrow = (datetime.today() + timedelta(days = 1)).strftime("%Y-%m-%d")
yesterday = (datetime.today() - timedelta(days = 1)).strftime("%Y-%m-%d")
day_before_yesterday = (datetime.today() - timedelta(days = 2)).strftime("%Y-%m-%d")

# Schedule = statsapi.schedule(start_date = day_before_yesterday, end_date = yesterday)
Schedule = statsapi.schedule(start_date = "2023-04-01", end_date = tomorrow)
Schedule_DataFrame = pd.json_normalize(Schedule)

game_id_list = list(Schedule_DataFrame["game_id"].drop_duplicates())
start = datetime.now()

In [187]:
Schedule_DataFrame[['game_datetime', 'game_date', 'home_probable_pitcher', 'away_probable_pitcher',
       'away_score', 'home_score',
       'current_inning', 'inning_state', 'status']].tail(2)

,game_datetime,game_date,home_probable_pitcher,away_probable_pitcher,away_score,home_score,current_inning,inning_state,status
709,2023-05-23T01:38:00Z,2023-05-22,,Tanner Houck,0,0,,,Scheduled
710,2023-05-23T01:40:00Z,2023-05-22,Luis Castillo,Kyle Muller,0,0,,,Scheduled


In [188]:
Schedule_DataFrame[['game_datetime', 'game_date', 'home_probable_pitcher', 'away_probable_pitcher',
       'away_score', 'home_score',
       'current_inning', 'inning_state', 'status']].head(2)

,game_datetime,game_date,home_probable_pitcher,away_probable_pitcher,away_score,home_score,current_inning,inning_state,status
0,2023-04-01T18:10:00Z,2023-04-01,Jose Urquidy,Lucas Giolito,4,6,9,Top,Final
1,2023-04-01T18:15:00Z,2023-04-01,Jack Flaherty,Kevin Gausman,1,4,9,Top,Final


In [33]:
Schedule_DataFrame.columns

Index(['game_id', 'game_datetime', 'game_date', 'game_type', 'status',
       'away_name', 'home_name', 'away_id', 'home_id', 'doubleheader',
       'game_num', 'home_probable_pitcher', 'away_probable_pitcher',
       'home_pitcher_note', 'away_pitcher_note', 'away_score', 'home_score',
       'current_inning', 'inning_state', 'venue_id', 'venue_name',
       'national_broadcasts', 'series_status', 'winning_team', 'losing_team',
       'winning_pitcher', 'losing_pitcher', 'save_pitcher', 'summary'],
      dtype='object')

In [65]:
_boxscores = pickle.load(open('boxscores.pkl', 'rb'))

{}

In [190]:
def get_boxscore_data(game_id, force_fetch = False):
    global _boxscores
    data = None
    if game_id not in _boxscores or force_fetch:
        try:
            print(f'calling statsapi.boxscore_data for game_id: {game_id}')
            data = statsapi.boxscore_data(game_id)
        except Exception:
            print(f'Exception: {Exception} while fetching boxscore')
        if data is not None:
            _boxscores[game_id] = data
    elif game_id in _boxscores:
        data = _boxscores[game_id]
    
    return data

def dump_boxscore_cache():
    global _boxscores
    pickle.dump(_boxscores, open('boxscores.pkl', 'wb'))

In [192]:
dump_boxscore_cache()

In [191]:
for i, game_id in enumerate(game_id_list):
    if i % 10 == 0:
        print(f'processing {i} out of {len(game_id_list)} game_id {game_id}')
    Game = Schedule_DataFrame[Schedule_DataFrame["game_id"] == game_id]

    home_team_name, home_team_id = Game["home_name"].iloc[0], Game["home_id"].iloc[0]
    away_team_name, away_team_id = Game["away_name"].iloc[0], Game["away_id"].iloc[0]

    home_probable_pitcher = Game["home_probable_pitcher"].iloc[0]
    away_probable_pitcher = Game["away_probable_pitcher"].iloc[0]
    
    game_boxscore = get_boxscore_data(game_id)

processing 0 out of 698 game_id 718760
calling statsapi.boxscore_data for game_id: 718760
calling statsapi.boxscore_data for game_id: 718759
calling statsapi.boxscore_data for game_id: 718758
calling statsapi.boxscore_data for game_id: 718752
calling statsapi.boxscore_data for game_id: 718753
calling statsapi.boxscore_data for game_id: 718762
calling statsapi.boxscore_data for game_id: 718757
calling statsapi.boxscore_data for game_id: 718761
calling statsapi.boxscore_data for game_id: 718756
calling statsapi.boxscore_data for game_id: 718754
processing 10 out of 698 game_id 718751
calling statsapi.boxscore_data for game_id: 718751
calling statsapi.boxscore_data for game_id: 718755
calling statsapi.boxscore_data for game_id: 718750
calling statsapi.boxscore_data for game_id: 718749
calling statsapi.boxscore_data for game_id: 718746
calling statsapi.boxscore_data for game_id: 718742
calling statsapi.boxscore_data for game_id: 718748
calling statsapi.boxscore_data for game_id: 718743
cal

In [83]:
_player_stats = pickle.load(open('player_stats.pkl', 'rb'))

In [104]:
def get_player_stat_data(player_id, force_fetch = False):
    global _player_stats
    data = None
    if player_id not in _player_stats or force_fetch:
        try:
            print(f'calling statsapi.player_stat_data for player_id: {player_id}')
            data = statsapi.player_stat_data(personId = player_id, group="hitting", type="yearByYear", sportId=1)
        except Exception:
            print(f'Exception: {Exception} while fetching player_stat_data')
        if data is not None:
            _player_stats[player_id] = data
    elif game_id in _boxscores:
        data = _player_stats[player_id]
    
    return data

def dump_player_stat_data_cache():
    global _player_stats
    pickle.dump(_player_stats, open('player_stats.pkl', 'wb'))

In [155]:
def get_matchup(game_boxscore, side):
    opposite_side = "away" if side == "home" else "home"
    all_side_players = game_boxscore[side]["players"]
    
    side_players_list = []
    for home_player in all_side_players:
        home_player = all_side_players[home_player]
        home_player_name, home_player_id = home_player["person"]["fullName"], home_player["person"]["id"]
        side_players_list.append({"name":home_player_name, "id":home_player_id})

    side_players_dataframe = pd.DataFrame(side_players_list)
    side_batting_lineup_ids = game_boxscore[side]["battingOrder"]
    side_batters = side_players_dataframe[side_players_dataframe["id"].isin(side_batting_lineup_ids)]
    valid_historical_season = int(Game["game_date"].iloc[0][0:4]) - 1
    side_batter_stats_list = []

    for side_batter in side_batters["id"]:
        side_batter_stats_data = get_player_stat_data(side_batter, force_fetch = False)
        if side_batter_stats_data is not None:
            side_batter_stats = side_batter_stats_data["stats"]
        else:
            print(f'Error while getting {side} batter {side_batter} stat')
            return None

        if len(side_batter_stats) == 0:
            print(f'{side} batter {side_batter} stat is empty')
            return None
        
        for historical_batter_stat in side_batter_stats:
            if historical_batter_stat["season"] == str(valid_historical_season):
                season_batter_stats = historical_batter_stat["stats"]
                season_batter_stats["name"] = side_batters["name"][side_batters["id"] == side_batter].iloc[0]
                season_batter_stats["id"] = side_batters["id"][side_batters["id"] == side_batter].iloc[0]
                
                side_batter_game_day_stats = all_side_players[f"ID{side_batter}"]["stats"]["batting"]
                
                if side_batter_game_day_stats["hits"] < 1:
                    hit_recorded = 0
                elif side_batter_game_day_stats["hits"] >= 1:
                    hit_recorded = 1
                    
                season_batter_stats["hit_recorded"] = hit_recorded
                side_batter_stats_list.append(season_batter_stats)
            
    side_team_batting_stats = pd.DataFrame(side_batter_stats_list).drop_duplicates(subset = "name", keep ="last")
    
    if len(side_team_batting_stats) < 1:
        print(f'{side} side_team_batting_stats is empty for game')
        return None
    
    opposing_pitcher_name, opposing_pitcher_id = Game[f"{opposite_side}_probable_pitcher"].iloc[0], Player_to_ID(Game[f"{opposite_side}_probable_pitcher"].iloc[0])
    
    side_batting_matchup = None
    if type(opposing_pitcher_id) != type(np.int64()):
        print(f'opposing_pitcher_id {opposing_pitcher_id} is not of int64 type')
        pass
    else:
        opposing_pitcher_stats_data = get_player_stat_data(opposing_pitcher_id, force_fetch = False)
        if opposing_pitcher_stats_data is not None:            
            opposing_pitcher_stats = pd.json_normalize(opposing_pitcher_stats_data["stats"], max_level = 0)
        else:
            print(f'Error while getting {side} opssosing pitcher {opposing_pitcher_id} stat')
            return None
        
        # If there is just no data from the API for this player
        if len(opposing_pitcher_stats) == 0:
            print(f'{side} opposing pitcher {opposing_pitcher_id} stat is empty')
            pass
        else:
            valid_opposing_pitcher_season_stats = opposing_pitcher_stats[opposing_pitcher_stats["season"] == str(valid_historical_season)]["stats"].drop_duplicates(keep = "last")
        
            # If there is no historical data for last season
            if len(valid_opposing_pitcher_season_stats) == 0:
                print(f'{side} valid opposing pitcher {opposing_pitcher_id} season stat is empty')
                pass
            else:
                opposing_pitcher_season_stats = valid_opposing_pitcher_season_stats.iloc[0]
                opposing_pitcher_season_stats["name"] = opposing_pitcher_name
                opposing_pitcher_season_stats["id"] = opposing_pitcher_id
                
                opposing_pitcher_stats_dataframe = pd.DataFrame([opposing_pitcher_season_stats])
                opposing_pitcher_stats = pd.concat([opposing_pitcher_stats_dataframe]*len(side_team_batting_stats))
                side_batting_matchup = pd.concat([opposing_pitcher_stats.reset_index(drop = True).add_prefix("pitching_"), side_team_batting_stats.reset_index(drop = True).add_prefix("batting_")], axis = 1)
            
                # =============================================================================
                # End of calculating the side for the batters
                # =============================================================================
    return side_batting_matchup

In [193]:
len(game_id_list)

698

In [197]:
game_matchups = []
for i, game_id in enumerate(game_id_list):
    print(f'processing {i} out of {len(game_id_list)} game_id {game_id}')
    Game = Schedule_DataFrame[Schedule_DataFrame["game_id"] == game_id]

    home_team_name, home_team_id = Game["home_name"].iloc[0], Game["home_id"].iloc[0]
    away_team_name, away_team_id = Game["away_name"].iloc[0], Game["away_id"].iloc[0]

    home_probable_pitcher = Game["home_probable_pitcher"].iloc[0]
    away_probable_pitcher = Game["away_probable_pitcher"].iloc[0]
    
    game_boxscore = get_boxscore_data(game_id)
    
    '''
    try:
        game_boxscore = statsapi.boxscore_data(game_id)
    except Exception:
        continue
    '''

    home_batting_matchup = get_matchup(game_boxscore, "home")
    away_batting_matchup = get_matchup(game_boxscore, "away")
    
    home_batting_matchup = home_batting_matchup if home_batting_matchup is not None else []
    away_batting_matchup = away_batting_matchup if away_batting_matchup is not None else []

    all_home_players = game_boxscore["home"]["players"]
    all_away_players = game_boxscore["away"]["players"]

    if (len(home_batting_matchup) == 0) and (len(away_batting_matchup) == 0):
        print(f'None both home_batting_matchup and away_batting_matchup for game_id {game_id}')
        continue
    elif (len(home_batting_matchup) >= 1) and (len(away_batting_matchup) == 0):
        print(f'None away_batting_matchup for game_id {game_id}')
        game_matchup = home_batting_matchup.reset_index(drop = True).copy()
    elif (len(home_batting_matchup) == 0) and (len(away_batting_matchup) >= 1):
        print(f'None home_batting_matchup for game_id {game_id}')
        game_matchup = away_batting_matchup.reset_index(drop = True).copy()
    elif (len(home_batting_matchup) >= 0) and (len(away_batting_matchup) >= 1):
        game_matchup = pd.concat([home_batting_matchup, away_batting_matchup], axis = 0).reset_index(drop = True)

    game_matchup["game_id"] = Game["game_id"].iloc[0]
    game_matchup["game_venue"] = Game["venue_name"].iloc[0]
    game_matchup["game_date"] = Game["game_date"].iloc[0]
    game_matchup["game_datetime"] = Game["game_datetime"].iloc[0]

    if Game["venue_name"].isin(list(Park_Data["Venue"])).iloc[0] == False:
        continue

    park_lat = Park_Data["latitude"][Park_Data["Venue"] == game_matchup["game_venue"].iloc[0]].iloc[0]
    park_lon = Park_Data["longitude"][Park_Data["Venue"] == game_matchup["game_venue"].iloc[0]].iloc[0]

    point_object = meteostat.Point(lat = park_lat, lon = park_lon)

    historical_weather = meteostat.Hourly(loc = point_object, start = (pd.to_datetime(game_matchup["game_date"].iloc[0])), end = (pd.to_datetime(game_matchup["game_date"].iloc[0]) + timedelta(days = 1)), timezone = "America/Chicago").fetch().reset_index()

    pre_game_weather = historical_weather[historical_weather["time"] <= (pd.to_datetime(Game["game_datetime"].iloc[0])).tz_convert("America/Chicago")]
    last_hour_weather = pre_game_weather.tail(1).copy()
    last_hour_weather["temp_f"] = last_hour_weather["temp"].apply(celsius_to_fahrenheit).iloc[0]

    game_temperature = last_hour_weather["temp_f"].iloc[0]
    game_matchup["temp"] = game_temperature
    game_matchups.append(game_matchup)

processing 0 out of 698 game_id 718760
home valid opposing pitcher 608337 season stat is empty
away valid opposing pitcher 664353 season stat is empty
None both home_batting_matchup and away_batting_matchup for game_id 718760
processing 1 out of 698 game_id 718759
home valid opposing pitcher 592332 season stat is empty
away valid opposing pitcher 656427 season stat is empty
None both home_batting_matchup and away_batting_matchup for game_id 718759
processing 2 out of 698 game_id 718758
home valid opposing pitcher 605540 season stat is empty
away valid opposing pitcher 657006 season stat is empty
None both home_batting_matchup and away_batting_matchup for game_id 718758
processing 3 out of 698 game_id 718752
home valid opposing pitcher 554430 season stat is empty
away valid opposing pitcher 543135 season stat is empty
None both home_batting_matchup and away_batting_matchup for game_id 718752
processing 4 out of 698 game_id 718753
home valid opposing pitcher 675911 season stat is empty
a

In [196]:
dump_player_stat_data_cache()

In [184]:
Hit_Real_Features = ["pitching_gamesPlayed","pitching_runs", "pitching_strikeOuts", "pitching_hits","pitching_id","pitching_name", "batting_gamesPlayed", "batting_runs","batting_strikeOuts", "batting_hits","batting_rbi", "batting_hit_recorded","batting_id","batting_name","temp" ,"game_id", "game_venue","game_date"]

In [198]:
game_matchup_dataframe = pd.concat(game_matchups)[Hit_Real_Features].dropna()
print(f"Unique Games: {len(game_matchup_dataframe['game_id'].drop_duplicates())}")

Unique Games: 73


In [200]:
game_matchup_dataframe

,pitching_gamesPlayed,pitching_runs,pitching_strikeOuts,pitching_hits,pitching_id,pitching_name,batting_gamesPlayed,batting_runs,batting_strikeOuts,batting_hits,batting_rbi,batting_hit_recorded,batting_id,batting_name,temp,game_id,game_venue,game_date
0,29,0,0,0,657376,Clarke Schmidt,140,71,89,127,62,1,642731,Thairo Estrada,71.06,718762,Yankee Stadium,2023-04-01
1,29,0,0,0,657376,Clarke Schmidt,134,57,100,104,70,1,592626,Joc Pederson,71.06,718762,Yankee Stadium,2023-04-01
2,29,0,0,0,657376,Clarke Schmidt,52,21,58,36,24,1,681584,David Villar,71.06,718762,Yankee Stadium,2023-04-01
3,29,0,0,0,657376,Clarke Schmidt,118,50,98,94,52,1,543063,Brandon Crawford,71.06,718762,Yankee Stadium,2023-04-01
4,29,0,0,0,657376,Clarke Schmidt,11,1,7,2,2,0,669369,Bryce Johnson,71.06,718762,Yankee Stadium,2023-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,27,0,0,0,621076,James Kaprielian,157,85,110,183,75,0,547989,Jose Abreu,77.72,718086,Minute Maid Park,2023-05-21
5,27,0,0,0,621076,James Kaprielian,141,103,87,158,57,0,514888,Jose Altuve,77.72,718086,Minute Maid Park,2023-05-21
6,27,0,0,0,621076,James Kaprielian,136,72,135,132,63,0,665161,Jeremy Pena,77.72,718086,Minute Maid Park,2023-05-21
7,27,0,0,0,621076,James Kaprielian,119,47,106,88,44,0,676801,Chas McCormick,77.72,718086,Minute Maid Park,2023-05-21


In [157]:
Hit_Real_Features = ["pitching_gamesPlayed","pitching_runs", "pitching_strikeOuts", "pitching_hits","pitching_id","pitching_name", "batting_gamesPlayed", "batting_runs","batting_strikeOuts", "batting_hits","batting_rbi", "batting_hit_recorded","batting_id","batting_name","temp" ,"game_id", "game_venue","game_date"]

In [159]:
game_matchup_dataframe = pd.concat(game_matchups)

In [163]:
game_matchup_dataframe[Hit_Real_Features].dropna()

,pitching_gamesPlayed,pitching_runs,pitching_strikeOuts,pitching_hits,pitching_id,pitching_name,batting_gamesPlayed,batting_runs,batting_strikeOuts,batting_hits,batting_rbi,batting_hit_recorded,batting_id,batting_name,temp,game_id,game_venue,game_date
0,31,42,202,86,675911,Spencer Strider,5,3,4,2,1,0,682626,Francisco Alvarez,59.00,718352,Citi Field,2023-05-01
1,31,42,202,86,675911,Spencer Strider,148,73,61,174,62,1,643446,Jeff McNeil,59.00,718352,Citi Field,2023-05-01
2,31,42,202,86,675911,Spencer Strider,11,4,8,7,5,1,683146,Brett Baty,59.00,718352,Citi Field,2023-05-01
3,31,42,202,86,675911,Spencer Strider,136,58,129,119,69,1,500871,Eduardo Escobar,59.00,718352,Citi Field,2023-05-01
4,31,42,202,86,675911,Spencer Strider,144,89,167,131,63,0,502054,Tommy Pham,59.00,718352,Citi Field,2023-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,27,0,0,0,621076,James Kaprielian,157,85,110,183,75,0,547989,Jose Abreu,77.72,718086,Minute Maid Park,2023-05-21
5,27,0,0,0,621076,James Kaprielian,141,103,87,158,57,0,514888,Jose Altuve,77.72,718086,Minute Maid Park,2023-05-21
6,27,0,0,0,621076,James Kaprielian,136,72,135,132,63,0,665161,Jeremy Pena,77.72,718086,Minute Maid Park,2023-05-21
7,27,0,0,0,621076,James Kaprielian,119,47,106,88,44,0,676801,Chas McCormick,77.72,718086,Minute Maid Park,2023-05-21


In [20]:
baseball_historical_matchup = game_matchup_dataframe

In [25]:
game_matchup_dataframe["batter_game"] = game_matchup_dataframe["batting_id"].astype(str) + game_matchup_dataframe["game_id"].astype(str)

In [27]:
game_matchup_dataframe = game_matchup_dataframe.drop("batter_game", axis = 1)

In [46]:
game_matchup_dataframe.to_pickle('game_matchup_dataframe.pkl')